*This is a Google Collab notebook*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Unzipping** the datset

In [ ]:
#Extracting diffusion dataset from Drive
import zipfile
zip_file_path = '/content/drive/MyDrive/Copy of diffusion_datasets.zip'
extracted_dir = '/content/diffusion'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)


In [ ]:
#Extracting GAN dataset from Drive
import zipfile
zip_file_path = '/content/drive/MyDrive/Copy of CNN_synth_testset.zip'
extracted_dir = '/content/gan'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
#Checking availability of gpu
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


TensorFlow version: 2.15.0
GPU Available: True


# Importing libraries

In [69]:
import numpy as np
import os
from PIL import Image
import cv2
#Tensorflow Libraries
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# **Level-1** of the architecture

Gathering data

In [ ]:
fake_images=[]
path="/content/diffusion/diffusion_datasets/dalle/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Reading image nd convert to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    fake_images.append(img)

In [ ]:
real_images=[]
path="/content/diffusion/diffusion_datasets/laion/0_real"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if(np.shape(img)==(256,256,3)):
        real_images.append(img)

In [ ]:
data=[]
labels=[]  # 0-Real 1-Fake

#Data with both real and fake images
data.extend(real_images)
data.extend(fake_images[:988])

#Labels for the data 0 and 1's
l1=[0]*988
l2=[1]*988
labels.extend(l1)
labels.extend(l2)

In [ ]:
#Splitting training data into training and validation using sklearn
#validation=20%
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Inserting data into training and validation folders

In [ ]:
# Function to add data in folder according to its class
def create_fld(images, labels, dest):
    i=0
    for image, label in zip(images, labels):
        i=i+1
        class_dir=os.path.join(dest, str(label))
        os.makedirs(class_dir,exist_ok=True) #Creating a directory or each class and when created it doesnt create again because exist=True condition passed
        os.chdir(class_dir)
        output_file=str(i)+".jpg"
        cv2.imwrite(output_file,image)

In [ ]:
train_directory = "/content/train_l1"
val_directory =  "/content/validation_l1"

#Creating empty to directory for Training and Validation
os.makedirs(train_directory)
os.makedirs(val_directory)

create_fld(X_train, y_train, train_directory)
create_fld(X_val, y_val, val_directory)


# **Creating Structure of the model**

**1) Preprocessing of Data:**

- Gaussian Blur.
- Rescaling, rotation, flipping.
- Preparing Training and Validation Generator.

**2) Building Model:**

- Importing weights from ResNet50 model (Pre-Trained Model).
- **Fine Tuning:**
  - Freezing first 150 layers (Neural network won't train back first 150 layers).
  - Unfreezing all other layers (Weights get updated for all layers after the 150th layer).
- Used Early Stopping to stop the training process when no significant improvement in loss is observed and retaining the best weights in each step.
- Loss function: Binary Cross Entropy.

**3) Model Structure:**

- `x = base_model(inputs, training=False)`: Base_model is a pre-trained convolutional neural network (CNN) ResNet-50, used as a feature extractor.
- `x = GlobalAveragePooling2D()(x)`: After extracting features from the base model, a global average pooling layer is applied. Global Average Pooling 2D computes the average value of each feature map across the entire spatial dimensions. This reduces the spatial dimensions to 1x1, effectively summarizing the information in each feature map.
- `x = Dense(1024, activation='relu')(x)`: This is a fully connected (dense) layer with 1024 units and ReLU (Rectified Linear Unit) activation function. The output of the global average pooling is connected to this dense layer, introducing non-linearity and allowing the network to learn complex patterns.
- `x = tf.keras.layers.Dropout(0.2)(x)`: Dropout is a regularization technique that helps prevent overfitting. It randomly sets a fraction of input units to zero at each update during training, which helps prevent the network from relying too much on any specific set of neurons. In this case, 20% of the units are dropped out (set to zero).
- `outputs = Dense(1, activation='sigmoid')(x)`: The final layer is a dense layer with a single unit and a sigmoid activation function. The output is a probability between 0 and 1.
  - 0: Real Image
  - 1: Fake Image


**Hyperparameters:**

- Learning Rate: 0.00001
- Epochs: 40
- Momentum: 0.9
- Dropout Layers: 0.2
- Optimizer: SGD (Stochastic Gradient Descent)


In [ ]:
def model_l1(train_dir,validation_dir):
    #Applying gaussian blur to each image
    def apply_gaussian_blur(image):
        sigma = 1.0
        return cv2.GaussianBlur(image, (0, 0), sigma)

    #Data Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
        horizontal_flip=True,
        shear_range=0.2,
        fill_mode='nearest',
        preprocessing_function=apply_gaussian_blur
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    #Training Data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='binary'
    )

    #Validation Data
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='binary'
    )

    inputs = tf.keras.Input(shape=(256, 256, 3)) #Input shape

    #Loading weights from Resnet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    print("Number of layers in the base model: ", len(base_model.layers))

    #Fine-Tuning
    fine_tune_at = 150

    #Freezing first 150 layers
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    #Unfreezing all layers ater 150
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True

    #Adding layers to the model
    x = base_model(inputs, training=False)  #Resnet50 weights
    x = GlobalAveragePooling2D()(x)         #Dimension reduction layer
    x = Dense(1024, activation='relu')(x)   #FCNN layer
    x = tf.keras.layers.Dropout(0.2)(x)     #Droping out few weights to remove dependency of model on any particular weight
    outputs = Dense(1, activation='sigmoid')(x) #Dense layer(o//p) layer 0 and 1

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    #Early stopping stops epochs if mofel not imporving in each step
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    #compiling the model
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    #Fitting the model nd training for 40 epochs
    history=model.fit(train_generator, epochs=40,
              validation_data=validation_generator,
              callbacks=[early_stopping_callback])

    #Returning model nd history
    return model,history

# **Training Phase**

In [ ]:
t="/content/train_l1"
v="/content/validation_l1"

#Calling the above function
mod_l1,hist_l1=model_l1(t,v)

Found 1580 images belonging to 2 classes.
Found 396 images belonging to 2 classes.
Number of layers in the base model:  175
Epoch 1/40
53/53 [==============================] - 41s 677ms/step - loss: 0.7140 - accuracy: 0.5222 - val_loss: 0.6940 - val_accuracy: 0.5101
Epoch 2/40
53/53 [==============================] - 31s 582ms/step - loss: 0.7197 - accuracy: 0.4956 - val_loss: 0.6900 - val_accuracy: 0.5455
Epoch 3/40
53/53 [==============================] - 31s 587ms/step - loss: 0.7079 - accuracy: 0.5190 - val_loss: 0.6892 - val_accuracy: 0.5631
Epoch 4/40
53/53 [==============================] - 32s 610ms/step - loss: 0.7081 - accuracy: 0.5089 - val_loss: 0.6881 - val_accuracy: 0.5707
Epoch 5/40
53/53 [==============================] - 32s 599ms/step - loss: 0.7091 - accuracy: 0.5203 - val_loss: 0.6867 - val_accuracy: 0.5581
Epoch 6/40
53/53 [==============================] - 31s 585ms/step - loss: 0.7126 - accuracy: 0.4880 - val_loss: 0.6871 - val_accuracy: 0.5631
Epoch 7/40
53/53 [

Saving the model

In [ ]:
mod_l1.save("/content/Level1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing level-1

In [ ]:
#Testing on glide
test_images=[]
test_label=[]
path="/content/diffusion/diffusion_datasets/glide_100_10/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(1)

In [ ]:
path="/content/diffusion/diffusion_datasets/glide_100_27/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(1)

In [ ]:
path="/content/diffusion/diffusion_datasets/glide_50_27/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(1)

In [ ]:
#Storing testing data into directory
test_directory = "/content/test"
os.makedirs(test_directory,exist_ok=True)
create_fld(test_images,test_label, test_directory)

In [ ]:
test_dir = "/content/test"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=30,
    class_mode='binary',
    shuffle=False
)

Found 3000 images belonging to 1 classes.


In [ ]:
#Evaluating our model
results = mod_l1.evaluate(test_generator)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

100/100 [==============================] - 13s 127ms/step - loss: 0.6085 - accuracy: 0.8680
Test Loss: 0.608487606048584
Test Accuracy: 0.8679999709129333


# **Level-2** of the architecture

Preparing data

In [ ]:
gan_images=[]
path="/content/gan/whichfaceisreal/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Reading each image nd converting into np array
    img=np.array(Image.open(os.path.join(path,i)))

    #If not a RGB image then discard it
    if len(np.shape(img))==3:
        new_size = (224, 224)
        img = cv2.resize(img, new_size)  #Resizing image st all images are of same dimension
        gan_images.append(img)

In [ ]:
np.shape(gan_images)

(1000, 224, 224, 3)

In [ ]:
dm_images=[]
path="/content/diffusion/diffusion_datasets/glide_100_10/1_fake"
l=os.listdir(path) #List containing path of all images in this folder

#Iterating into each image
for i in l:
    #Reading each image nd converting into np array
    img=np.array(Image.open(os.path.join(path,i)))

    #If not a RGB image then discard it
    if len(np.shape(img))==3:
      dm_images.append(img)

In [ ]:
np.shape(dm_images)

(1000, 256, 256, 3)

In [ ]:
#Data containing both gan nd dm images
data=[]
labels=[]  # 0-Dm 1-Gan
data.extend(dm_images)
data.extend(gan_images)

#Labels for both gan and dm images
l1=[0]*len(dm_images)
l2=[1]*len(gan_images)
labels.extend(l1)
labels.extend(l2)

In [ ]:
print("Total images:",len(data),"Dimension of each image:",np.shape(data[0]))
print(np.shape(labels))

Total images: 2000 Dimension of each image: (256, 256, 3)
(2000,)


In [ ]:
#Splitting data into train nd validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
train_d = "/content/train_l2"
val_d =  "/content/validation_l2"

#Creating empty to directory for Training and Validation
os.makedirs(train_d)
os.makedirs(val_d)

create_fld(X_train, y_train, train_d)
create_fld(X_val, y_val, val_d)


# **Creating Structure of Model**

**1) Preprocessing of Data:**

- Gaussian Blur.
- Rescaling, rotation, flipping.
- Preparing Training and Validation Generator.

**2) Building Model:**

- Importing weights from ResNet50 model (Pre-Trained Model).
- **Fine Tuning:**
  - Freezing first 100 layers (Neural network won't train back first 100 layers).
  - Unfreezing all other layers (Weights get updated for all layers after the 100th layer).
- Used Early Stopping to stop the training process when no significant improvement in loss is observed and retaining the best weights in each step.
- Loss function: Binary Cross Entropy.

**3) Model Structure:**

- `x = base_model(inputs, training=False)`: Base_model is a pre-trained convolutional neural network (CNN) ResNet-50, used as a feature extractor.
- `x = GlobalAveragePooling2D()(x)`: After extracting features from the base model, a global average pooling layer is applied. Global Average Pooling 2D computes the average value of each feature map across the entire spatial dimensions. This reduces the spatial dimensions to 1x1, effectively summarizing the information in each feature map.
- `x = Dense(1024, activation='relu')(x)`: This is a fully connected (dense) layer with 1024 units and ReLU (Rectified Linear Unit) activation function. The output of the global average pooling is connected to this dense layer, introducing non-linearity and allowing the network to learn complex patterns.
- `x = tf.keras.layers.Dropout(0.2)(x)`: Dropout is a regularization technique that helps prevent overfitting. It randomly sets a fraction of input units to zero at each update during training, which helps prevent the network from relying too much on any specific set of neurons. In this case, 20% of the units are dropped out (set to zero).
- `outputs = Dense(1, activation='sigmoid')(x)`: The final layer is a dense layer with a single unit and a sigmoid activation function. The output is a probability between 0 and 1.
  - 0: Real Image
  - 1: Fake Image


**Learning Rate Schedule: Exponential Decay**

- **Initial Learning Rate:** \(1 \times 10^{-4}\) (0.0001)
  - The starting learning rate at the beginning of training.

- **Decay Steps:** 10,000
  - After every 10,000 steps, the learning rate will be updated.

- **Decay Rate:** 0.9
  - The rate at which the learning rate will decay, multiplied at each decay step.

By employing this exponentially decaying learning rate schedule, the model takes larger steps initially and gradually decreases the learning rate as training progresses, aiding in faster convergence and fine-tuning

**Hyperparameters:**

- Epochs: 40
- Momentum: 0.9
- Dropout Layers: 0.2
- Optimizer: SGD (Stochastic Gradient Descent)


In [ ]:
def model_l2(train_dir,validation_dir,tune,epoc):
    #Applying gaussian blur to each image
    def apply_gaussian_blur(image):
        sigma = 1.0
        return cv2.GaussianBlur(image, (0, 0), sigma)

    #DATA Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
        horizontal_flip=True,
        shear_range=0.2,
        fill_mode='nearest',
        preprocessing_function=apply_gaussian_blur
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    #Training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='binary'
    )
    #VAlidation data
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='binary'
    )
    inputs = tf.keras.Input(shape=(256, 256, 3)) #Input shape

    #Loading weights from Resnet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    print("Number of layers in the base model: ", len(base_model.layers))

    #Fine-Tuning
    fine_tune_at = tune

    #Freezing first 100 layers
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    #Unfreezing all layers ater 100
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True

    #Adding layers to the model
    x = base_model(inputs, training=False)  #Resnet50 weights
    x = GlobalAveragePooling2D()(x)         #Dimension reduction layer
    x = Dense(1024, activation='relu')(x)   #FCNN layer
    x = tf.keras.layers.Dropout(0.2)(x)     #Droping out few weights to remove dependency of model on any particular weight
    outputs = Dense(1, activation='sigmoid')(x) #Dense layer(o//p) layer 0 and 1

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    #Learning Rate in terms of exponential Decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
    )
    #Early stopping stops epochs if mofel not imporving in each step
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    #compiling the model
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    #Fitting the model nd training for 40 epochs
    history=model.fit(train_generator, epochs=epoc,
              validation_data=validation_generator,
              callbacks=[early_stopping_callback])

    #Returning model nd history
    return model,history

# **Training** phase

In [ ]:
t="/content/train_l2"
v="/content/validation_l2"
#Training the model
mod_l2,hist_l2=model_l2(t,v,100,40)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Number of layers in the base model:  175
Epoch 1/40
54/54 [==============================] - 45s 612ms/step - loss: 0.6936 - accuracy: 0.5431 - val_loss: 0.6638 - val_accuracy: 0.5150
Epoch 2/40
54/54 [==============================] - 33s 615ms/step - loss: 0.7032 - accuracy: 0.5169 - val_loss: 0.6831 - val_accuracy: 0.4950
Epoch 3/40
54/54 [==============================] - 33s 596ms/step - loss: 0.6690 - accuracy: 0.5838 - val_loss: 0.6101 - val_accuracy: 0.7200
Epoch 4/40
54/54 [==============================] - 32s 599ms/step - loss: 0.6123 - accuracy: 0.6737 - val_loss: 0.6488 - val_accuracy: 0.5550
Epoch 5/40
54/54 [==============================] - 33s 608ms/step - loss: 0.5849 - accuracy: 0.6662 - val_loss: 0.6206 - val_accuracy: 0.8325
Epoch 6/40
54/54 [==============================] - 33s 611ms/step - loss: 0.5899 - accuracy: 0.6837 - val_loss: 0.6921 - val_accuracy: 0.4975
Epoch 7/40
54/54 [

Saving the model

In [ ]:
mod_l2.save("/content/Level2_updated.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing Level-2

In [ ]:
test_images=[]
test_label=[]
path="/content/diffusion/diffusion_datasets/guided/1_fake"
l=os.listdir(path)
for i in l:
    img=np.array(Image.open(os.path.join(path,i)))
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(0)

In [ ]:
path="/content/gan/stargan/1_fake"
l=os.listdir(path)
for i in l:
    img=np.array(Image.open(os.path.join(path,i)))
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(1)

In [ ]:
test_directory = "/content/test1"
os.makedirs(test_directory,exist_ok=True)
create_fld(test_images,test_label, test_directory)

In [ ]:
test_dir = "/content/test1"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=30,
    class_mode='binary',
    shuffle=False
)

Found 2999 images belonging to 2 classes.


In [ ]:
#Testing thr model
results = mod_l2.evaluate(test_generator)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

100/100 [==============================] - 12s 123ms/step - loss: 0.2682 - accuracy: 0.9300
Test Loss: 0.2682065963745117
Test Accuracy: 0.9299766421318054


# **Level-3 of the Architecture(DM)**

Detecting to which class the fake image belongs to
Classes: Dalle Glide LDM Guided

Gathering data

In [ ]:
dalle=[]
path="/content/diffusion/diffusion_datasets/dalle/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[:800]

#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if(np.shape(img)==(256,256,3)):
        dalle.append(img)
np.shape(dalle)

(800, 256, 256, 3)

In [ ]:
glide=[]
path="/content/diffusion/diffusion_datasets/glide_100_10/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[:800]

#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if(np.shape(img)==(256,256,3)):
        glide.append(img)
np.shape(glide)

(800, 256, 256, 3)

In [ ]:
guided=[]
path="/content/diffusion/diffusion_datasets/guided/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[:800]

#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if(np.shape(img)==(256,256,3)):
        guided.append(img)
np.shape(guided)

(800, 256, 256, 3)

In [ ]:
ldm=[]
path="/content/diffusion/diffusion_datasets/ldm_100/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[:800]

#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if(np.shape(img)==(256,256,3)):
        ldm.append(img)
np.shape(ldm)

(800, 256, 256, 3)

In [ ]:
#Adding all images to data
data=[]
data.extend(dalle)
data.extend(glide)
data.extend(guided)
data.extend(ldm)
np.shape(data)

(3200, 256, 256, 3)

In [ ]:
#Creating labels
labels=[0]*800
labels.extend([1]*800)
labels.extend([2]*800)
labels.extend([3]*800)
np.shape(labels)

(3200,)

Splitting data

In [ ]:
#Splitting training data into training and validation using sklearn
#validation=10%
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

Inserting data to folders

In [ ]:
train_directory = "/content/train_l3a"
val_directory =  "/content/validation_l3a"

#Creating empty to directory for Training and Validation
os.makedirs(train_directory)
os.makedirs(val_directory)

create_fld(X_train, y_train, train_directory)
create_fld(X_val, y_val, val_directory)


# **Model Architecture for Level-3**

In [86]:
def model_l3a(train_dir,validation_dir,tune,epoc):
    #DATA Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
        horizontal_flip=True,
        shear_range=0.2,
        fill_mode='nearest',
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    #Training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='categorical'
    )
    #VAlidation data
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(256, 256),
        batch_size=30,
        class_mode='categorical'
    )
    inputs = tf.keras.Input(shape=(256, 256, 3)) #Input shape

    #Loading weights from Resnet50
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    print("Number of layers in the base model: ", len(base_model.layers))

    #Fine-Tuning
    fine_tune_at = tune

    #Freezing first 100 layers
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    #Unfreezing all layers ater 100
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True

    #Adding layers to the model
    x = base_model(inputs, training=False)  #Resnet50 weights
    x = GlobalAveragePooling2D()(x)         #Dimension reduction layer
    x = Dense(1024, activation='relu')(x)   #FCNN layer
    x = tf.keras.layers.Dropout(0.2)(x)     #Droping out few weights to remove dependency of model on any particular weight
    outputs = Dense(4, activation='softmax')(x) #Dense layer(o//p) layer 0 and 1

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    #Learning Rate in terms of exponential Decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
    )
    #Early stopping stops epochs if mofel not imporving in each step
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    #compiling the model
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    #Fitting the model nd training for 40 epochs
    history=model.fit(train_generator, epochs=epoc,
              validation_data=validation_generator,
              callbacks=[early_stopping_callback])

    #Returning model nd history
    return model,history

# Training Level-3

In [87]:
t="/content/train_l3a"
v="/content/validation_l3a"
#Training the model
mod_l3a,hist_l3a=model_l3a(t,v,100,40)

Found 2880 images belonging to 4 classes.
Found 320 images belonging to 4 classes.
Number of layers in the base model:  175
Epoch 1/40
96/96 [==============================] - 65s 588ms/step - loss: 1.3537 - accuracy: 0.3257 - val_loss: 1.2482 - val_accuracy: 0.4500
Epoch 2/40
96/96 [==============================] - 56s 587ms/step - loss: 1.2632 - accuracy: 0.3837 - val_loss: 1.2239 - val_accuracy: 0.4875
Epoch 3/40
96/96 [==============================] - 56s 581ms/step - loss: 1.2383 - accuracy: 0.4056 - val_loss: 1.3034 - val_accuracy: 0.3375
Epoch 4/40
96/96 [==============================] - 55s 573ms/step - loss: 1.2251 - accuracy: 0.4205 - val_loss: 1.2884 - val_accuracy: 0.4313
Epoch 5/40
96/96 [==============================] - 57s 593ms/step - loss: 1.2144 - accuracy: 0.4292 - val_loss: 1.1770 - val_accuracy: 0.3844
Epoch 6/40
96/96 [==============================] - 54s 559ms/step - loss: 1.2115 - accuracy: 0.4292 - val_loss: 1.1497 - val_accuracy: 0.4750
Epoch 7/40
96/96 [

Saving the model

In [ ]:
mod_l3a.save("/content/Level3a.h5")

# **Testing Level-3**

In [ ]:
test_images=[]
test_label=[]
path="/content/diffusion/diffusion_datasets/dalle/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[800:]
#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(0)

In [ ]:
path="/content/diffusion/diffusion_datasets/glide_100_10/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[800:]
#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(1)

In [ ]:
path="/content/diffusion/diffusion_datasets/guided/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[800:]
#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(2)

In [ ]:
path="/content/diffusion/diffusion_datasets/ldm_100/1_fake"
l=os.listdir(path) #List containing path of all images in this folder
l.sort()
l1=l[800:]
#Iterating into each image
for i in l1:
    #Opening the image nd converting to numpy array
    img=np.array(Image.open(os.path.join(path,i)))
    #If not a RGB image, then discarding it
    if len(np.shape(img))==3:
        test_images.append(img)
        test_label.append(3)

In [76]:
test_directory = "/content/test2"
os.makedirs(test_directory,exist_ok=True)
create_fld(test_images,test_label, test_directory)

In [79]:
test_dir = "/content/test2"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=30,
    class_mode='categorical',
    shuffle=False
)

Found 800 images belonging to 4 classes.


In [88]:
#Testing thr model
results = mod_l3a.evaluate(test_generator)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

27/27 [==============================] - 3s 123ms/step - loss: 0.9937 - accuracy: 0.5500
Test Loss: 0.9936790466308594
Test Accuracy: 0.550000011920929
